Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10


def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
print(dir(tf))
graph = tf.Graph()
with graph.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

['AggregationMethod', 'Assert', 'AttrValue', 'COMPILER_VERSION', 'ConditionalAccumulator', 'ConditionalAccumulatorBase', 'ConfigProto', 'DType', 'DeviceSpec', 'Dimension', 'Event', 'FIFOQueue', 'FixedLenFeature', 'FixedLenSequenceFeature', 'FixedLengthRecordReader', 'GIT_VERSION', 'GPUOptions', 'GRAPH_DEF_VERSION', 'GRAPH_DEF_VERSION_MIN_CONSUMER', 'GRAPH_DEF_VERSION_MIN_PRODUCER', 'Graph', 'GraphDef', 'GraphKeys', 'GraphOptions', 'HistogramProto', 'IdentityReader', 'IndexedSlices', 'InteractiveSession', 'LogMessage', 'MetaGraphDef', 'NameAttrList', 'NoGradient', 'NodeDef', 'NotDifferentiable', 'OpError', 'Operation', 'OptimizerOptions', 'PaddingFIFOQueue', 'Print', 'PriorityQueue', 'QUANTIZED_DTYPES', 'QueueBase', 'RandomShuffleQueue', 'ReaderBase', 'RegisterGradient', 'RunMetadata', 'RunOptions', 'Session', 'SessionLog', 'SparseConditionalAccumulator', 'SparseFeature', 'SparseTensor', 'SparseTensorValue', 'Summary', 'TFRecordReader', 'Tensor', 'TensorArray', 'TensorInfo', 'TensorShap

Let's run this computation and iterate:

In [5]:
num_steps = 801


def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if step % 100 == 0:
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized


Loss at step 0: 18.641151
Training accuracy: 6.3%
Validation accuracy: 8.2%


Loss at step 100: 2.126178
Training accuracy: 72.7%
Validation accuracy: 70.5%


Loss at step 200: 1.750034
Training accuracy: 75.5%
Validation accuracy: 73.0%


Loss at step 300: 1.535720
Training accuracy: 76.5%
Validation accuracy: 74.1%


Loss at step 400: 1.385153
Training accuracy: 77.4%
Validation accuracy: 74.6%


Loss at step 500: 1.270921
Training accuracy: 78.2%
Validation accuracy: 74.8%


Loss at step 600: 1.180327
Training accuracy: 78.6%
Validation accuracy: 75.1%


Loss at step 700: 1.106323
Training accuracy: 79.0%
Validation accuracy: 75.2%


Loss at step 800: 1.044599
Training accuracy: 79.5%
Validation accuracy: 75.4%
Test accuracy: 82.3%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

In [6]:
batch_size = 524

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run it:

In [8]:
num_steps = 9001

with tf.Session(graph=graph, config=tf.ConfigProto(log_device_placement=True)) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 14.519587
Minibatch accuracy: 11.5%
Validation accuracy: 15.9%


Minibatch loss at step 500: 1.529944
Minibatch accuracy: 78.1%
Validation accuracy: 75.4%


Minibatch loss at step 1000: 1.248431
Minibatch accuracy: 76.5%
Validation accuracy: 76.8%


Minibatch loss at step 1500: 1.098871
Minibatch accuracy: 76.7%
Validation accuracy: 77.5%


Minibatch loss at step 2000: 0.959322
Minibatch accuracy: 80.2%
Validation accuracy: 78.4%


...

Minibatch loss at step 7000: 0.720788
Minibatch accuracy: 81.1%
Validation accuracy: 80.7%


Minibatch loss at step 7500: 0.591932
Minibatch accuracy: 84.5%
Validation accuracy: 80.7%


Minibatch loss at step 8000: 0.730762
Minibatch accuracy: 80.7%
Validation accuracy: 80.9%


Minibatch loss at step 8500: 0.739019
Minibatch accuracy: 82.3%
Validation accuracy: 81.1%


Minibatch loss at step 9000: 0.877589
Minibatch accuracy: 77.5%
Validation accuracy: 81.1%
Test accuracy: 88.2%


---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---

In [22]:
nodes = 1024
batch_size = 124
num_steps = 90000
graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, nodes]))
    biases_1 = tf.zeros([nodes])
    weights_2 = tf.Variable(tf.truncated_normal([nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    hidden_layer_in = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer_out = tf.nn.relu(hidden_layer_in)
    logits = tf.matmul(hidden_layer_out, weights_2) + biases_2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)

    valid_hidden_layer_in = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    valid_hidden_layer_out = tf.nn.relu(valid_hidden_layer_in)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden_layer_out, weights_2) + biases_2)

    test_hidden_layer_in = tf.matmul(tf_test_dataset, weights_1) + biases_1
    test_hidden_layer_out = tf.nn.relu(test_hidden_layer_in)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden_layer_out, weights_2) + biases_2)

In [23]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if step % 500 == 0:
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 316.876068
Minibatch accuracy: 8.9%
Validation accuracy: 25.6%


Minibatch loss at step 500: 12.070775
Minibatch accuracy: 80.6%
Validation accuracy: 80.3%


Minibatch loss at step 1000: 31.546661
Minibatch accuracy: 74.2%
Validation accuracy: 78.7%


Minibatch loss at step 1500: 8.888353
Minibatch accuracy: 75.0%
Validation accuracy: 80.5%


Minibatch loss at step 2000: 14.184120
Minibatch accuracy: 76.6%
Validation accuracy: 79.2%


...

Minibatch loss at step 88000: 0.501006
Minibatch accuracy: 97.6%
Validation accuracy: 85.6%


Minibatch loss at step 88500: 0.395098
Minibatch accuracy: 96.0%
Validation accuracy: 86.1%


Minibatch loss at step 89000: 7.971159
Minibatch accuracy: 93.5%
Validation accuracy: 85.5%


Minibatch loss at step 89500: 2.095283
Minibatch accuracy: 94.4%
Validation accuracy: 84.0%


Test accuracy: 92.5%
